# CMSC 394 survey data analysis NEW
## Christian Roncal

# Dependencies

In [1]:
import pandas as pd
import numpy as np

# Load and clean up the data

In [2]:
df = pd.read_csv('surveyresponses.csv')

In [3]:
df.head()

,Timestamp,Did you graduate from a highschool under the Prince George's County Public Schools system?,What year do you expect to receive your bachelors (if in combined BS/MS programs when did you finish your undergrad requirements)?,What year did you graduate from high school?,How many summer semesters have you taken?,What was your high school GPA?,What is your first semester college GPA?,"How many ""pre-requisite classes"" did you have to take? e.g. (Math115 - Precalculus to take Math140 - Calculus 1)",How many times have you switched majors?,Answer whether you agree/disagree with the following statements [I wasn't sure what I will do after high school],Answer whether you agree/disagree with the following statements [I felt uninterested in my high school coursework],Answer whether you agree/disagree with the following statements [My high school informed me about the military career path],Answer whether you agree/disagree with the following statements [My high school informed me about vocational career paths],Answer whether you agree/disagree with the following statements [My high school informed me about IT certifications]
0,4/24/2019 23:28:18,Yes,Spring 2019,2015,3,3.500,3.800,3,0,Disagree,Agree,Disagree,Disagree,Disagree
1,4/24/2019 23:33:04,No,Spring 2020,2017,1,3.760,3.947,0,0,Agree,Agree,Disagree,Disagree,Disagree
2,4/24/2019 23:34:46,No,Spring 2020,2016,4,4.778,2.987,0,1,Agree,Agree,Agree,Agree,Agree
3,4/24/2019 23:35:23,No,Spring 2020,2015,0,3.700,3.900,3,0,Agree,Agree,Disagree,Agree,Agree
4,4/24/2019 23:37:05,No,Spring 2020,2016,0,3.750,3.750,2,1,Agree,Agree,Disagree,Agree,Disagree


### Rename columns
1. pg - graduated from pgcps 1/0 (yes/no)
1. cgrad -college grad year semester
1. hsgrad - hs grad year
1. nsummer - number of summer semesters
1. nremedial - number of "prereq classes"
1. hsgpa - hs gpa
1. cgpa - first semester college gpa
1. nswitch - number of major switches
1. unsure - unsure of college plans
1. uninterested - unintersted in hs schoolwork
1. mil - military path known
1. voc - vocational path known
1. it - it path known

In [4]:
df.columns = ['timestamp', 'pg', 'cgrad', 'hsgrad', 'nsummer', 'hsgpa', 'cgpa', 'nremedial', 'nswitch', 'unsure', 'uninterested', 'mil', 'voc', 'it']

In [5]:
df = df.drop(columns='timestamp')
df.head()

,pg,cgrad,hsgrad,nsummer,hsgpa,cgpa,nremedial,nswitch,unsure,uninterested,mil,voc,it
0,Yes,Spring 2019,2015,3,3.500,3.800,3,0,Disagree,Agree,Disagree,Disagree,Disagree
1,No,Spring 2020,2017,1,3.760,3.947,0,0,Agree,Agree,Disagree,Disagree,Disagree
2,No,Spring 2020,2016,4,4.778,2.987,0,1,Agree,Agree,Agree,Agree,Agree
3,No,Spring 2020,2015,0,3.700,3.900,3,0,Agree,Agree,Disagree,Agree,Agree
4,No,Spring 2020,2016,0,3.750,3.750,2,1,Agree,Agree,Disagree,Agree,Disagree


### Fill in NaNs and numericalize

In [6]:
df = df.fillna('') # replaces all NaNs with empty string

### Numericalize

It's easier to deal with numbers when computationally analyzing data so let's only deal with numbers

In [7]:
agree_map = {'Yes': 1, 'No': 0, 'Agree':1, 'Disagree':0}
def numericalize(x):
    try:
        return agree_map[x]
    except:
        print(x)

In [8]:
col_names = ['pg', 'unsure', 'uninterested', 'mil', 'voc', 'it']
df[col_names].head()

,pg,unsure,uninterested,mil,voc,it
0,Yes,Disagree,Agree,Disagree,Disagree,Disagree
1,No,Agree,Agree,Disagree,Disagree,Disagree
2,No,Agree,Agree,Agree,Agree,Agree
3,No,Agree,Agree,Disagree,Agree,Agree
4,No,Agree,Agree,Disagree,Agree,Disagree


In [9]:
for col in col_names:
    df[col] = df[col].apply(numericalize)

In [10]:
df.head()

,pg,cgrad,hsgrad,nsummer,hsgpa,cgpa,nremedial,nswitch,unsure,uninterested,mil,voc,it
0,1,Spring 2019,2015,3,3.500,3.800,3,0,0,1,0,0,0
1,0,Spring 2020,2017,1,3.760,3.947,0,0,1,1,0,0,0
2,0,Spring 2020,2016,4,4.778,2.987,0,1,1,1,1,1,1
3,0,Spring 2020,2015,0,3.700,3.900,3,0,1,1,0,1,1
4,0,Spring 2020,2016,0,3.750,3.750,2,1,1,1,0,1,0


### Creating new features

nsems = grad year - hs grad year + 1 (if fall) + nsummer

In [11]:
def nsems(x):
    szn, year = x['cgrad'].split()
    year = int(year)
    fallszn = False if szn == 'Spring' else True
    
    res = year - x['hsgrad'] + x['nsummer']
    res += 1 if fallszn else 0
    return res

In [12]:
df['nsems'] = df.apply(nsems, axis=1)

In [13]:
df.head()

,pg,cgrad,hsgrad,nsummer,hsgpa,cgpa,nremedial,nswitch,unsure,uninterested,mil,voc,it,nsems
0,1,Spring 2019,2015,3,3.500,3.800,3,0,0,1,0,0,0,7
1,0,Spring 2020,2017,1,3.760,3.947,0,0,1,1,0,0,0,4
2,0,Spring 2020,2016,4,4.778,2.987,0,1,1,1,1,1,1,8
3,0,Spring 2020,2015,0,3.700,3.900,3,0,1,1,0,1,1,5
4,0,Spring 2020,2016,0,3.750,3.750,2,1,1,1,0,1,0,4


alt score = "degree of college alternative knowledge (ie number of alternative paths shown)" $\sum [\text{unintersted, mil, voc}]$

In [14]:
def altscore(x):
    return x['mil'] + x['voc'] + x['it']

In [15]:
df['altscore'] = df.apply(altscore, axis=1)

In [16]:
df.head()

,pg,cgrad,hsgrad,nsummer,hsgpa,cgpa,nremedial,nswitch,unsure,uninterested,mil,voc,it,nsems,altscore
0,1,Spring 2019,2015,3,3.500,3.800,3,0,0,1,0,0,0,7,0
1,0,Spring 2020,2017,1,3.760,3.947,0,0,1,1,0,0,0,4,0
2,0,Spring 2020,2016,4,4.778,2.987,0,1,1,1,1,1,1,8,3
3,0,Spring 2020,2015,0,3.700,3.900,3,0,1,1,0,1,1,5,2
4,0,Spring 2020,2016,0,3.750,3.750,2,1,1,1,0,1,0,4,1


In [17]:
df.to_csv('num_394.csv', index=False)

In [18]:
t = pd.read_csv('num_394.csv')
len(t)

50

In [19]:
df_clean = df.drop(columns=['cgrad', 'hsgrad', 'nsummer'])
df_clean.head()

,pg,hsgpa,cgpa,nremedial,nswitch,unsure,uninterested,mil,voc,it,nsems,altscore
0,1,3.500,3.800,3,0,0,1,0,0,0,7,0
1,0,3.760,3.947,0,0,1,1,0,0,0,4,0
2,0,4.778,2.987,0,1,1,1,1,1,1,8,3
3,0,3.700,3.900,3,0,1,1,0,1,1,5,2
4,0,3.750,3.750,2,1,1,1,0,1,0,4,1


In [20]:
df_clean.to_csv('394clean.csv', index=False)